### Tratamentos da Base Original

- Notebook criado para tratar dados de input antes de passar para o CDM OMOP. Ex: como criar campos de id

## Configurações PySpark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd


spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 17:20:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Configurações Pandas

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

## Leitura da base original

In [3]:
df_input = (spark
            .read
            .option('nullValue', 'NA')
            .csv('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/Karine/Banco/basefinal_gest_limp.csv', header=True))

### Métricas de leitura

In [4]:
# quantidade de registros com id_cidacs_mae_sinasc não nulos na base completa
qtd_registros_base_com_id_not_null = df_input.select('id_cidacs_mae_sinasc').filter(F.col('id_cidacs_mae_sinasc').isNotNull()).count()
print(f'Quantidade de registros com id_cidacs_mae_sinasc não nulos: {qtd_registros_base_com_id_not_null}')

qtd_id_mae_cidacs_not_null = df_input.filter(F.col('id_cidacs_mae_sinasc').isNotNull()).select("id_cidacs_mae_sinasc").distinct().count()
print(f'Quantidades de id_cidacs_mae_sinasc distintos: {qtd_id_mae_cidacs_not_null}')

Quantidade de registros com id_cidacs_mae_sinasc não nulos: 16609493


Quantidades de id_cidacs_mae_sinasc distintos: 12671004


#### 1. Mostrando que existem id_cidacs_sinasc_v4 que se repetem

 - podem ser gêmeos e também registros duplicados como é o caso abaixo

In [5]:
# mostrando dados duplicados para o mesmo infant
#************* Tratar: excluir nascimentos multiplos, pegar o último dtnotific_mae
df_input.filter(F.col('id_cidacs_sinasc_v5')=='444164246').toPandas()

,apgar1_sinasc,apgar5_sinasc,cod_raca_cor_pessoa_eq,codanomal_sinasc,codmunres_sinasc,consprenat_sinasc,consultas_sinasc,dt_notific_mae,dtnasc_sinasc,dtnascmae_sinasc,dtobito_sim.y,escmae_sinasc,estcivmae_sinasc,gravidez_sinasc,id_agravo_mae,id_cidacs_mae_sinasc,id_cidacs_sinasc_v5,idademae_sinasc,idanomal_sinasc,mesprenat_sinasc,parto_sinasc,peso_sinasc,qtdfilmort_sinasc,qtdfilvivo_sinasc,racacormae_sinasc,semagestac_sinasc,sexo_sinasc,tpapresent_sinasc,tpconfirma_mae,tpesquema_mae,tpevidenci_mae,tpteste1_mae,tra_dt_sc,marc_pbf_eq_v4,id_cidacs_pop100_v4,ind_mae_sinasc_cidacs,dtultmenst_sinasc,qtdgestant_sinasc
0,5,7,4,None,250750,99,88,2012-07-09,2014-08-09,1994-03-19,None,3,5,1,O981,877053219,444164246,20,2,99,1,3115,0,1,4,40,1,1,1,3,88,1,None,1,1236324582,0,None,1
1,5,7,4,None,250750,99,88,2020-01-03,2014-08-09,1994-03-19,None,3,5,1,O981,877053219,444164246,20,2,99,1,3115,0,1,4,40,1,1,1,5,88,1,None,1,1236324582,0,None,1


In [6]:
# Mostrando quais ids e a sua quantidade/gêmeos tem duplicidade
count_registros_duplicados_infant = df_input.groupBy('id_cidacs_sinasc_v5').count().filter(F.col('count')>1).count()
print(f'Quantidade de infants que aparecem mais de uma vez na base: {count_registros_duplicados_infant}')

Quantidade de infants que aparecem mais de uma vez na base: 17544


# Tratamentos

### 1. Removendo registros com id_cidacs_mae_sinasc = NA/Nulo

In [7]:
df = df_input.filter(F.col('id_cidacs_mae_sinasc').isNotNull())
print(f'Quantidade de id_cidacs_mae_sinasc nulos: {df_input.filter(F.col("id_cidacs_mae_sinasc").isNull()).count()}')
print(f'Quantidade de id_cidacs_mae_sinasc distintos em toda a base: {df.select("id_cidacs_mae_sinasc").distinct().count()}')

Quantidade de id_cidacs_mae_sinasc nulos: 0


Quantidade de id_cidacs_mae_sinasc distintos em toda a base: 12671004


###  2. Criando Visit Occurrence ID

- Cada registro não duplicado deve ser equivalente à uma visita, ou seja, ter um visit_occurrence_id próprio
- Obs.: Nesse caso ainda não estamos removendo os duplicados (verificar no item 1. da sessão 'Métricas de leitura'

In [8]:
df = df.withColumn('visit_occurrence_id', F.row_number().over(Window.orderBy('id_cidacs_mae_sinasc', 'dtnasc_sinasc')))

### 2.1 Criando Visit Occurrence ID do bebe

In [9]:
# Encontrando o maior person_id da mãe
max_mother_visit = df.agg(F.max(F.col('visit_occurrence_id')).alias('max_value')).collect()[0]['max_value']
max_mother_visit

25/05/06 13:56:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:56:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:56:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:56:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


16609493

In [10]:
df = df.withColumn('visit_occurrence_id_infant', F.row_number().over(Window.orderBy('id_cidacs_mae_sinasc', 'dtnasc_sinasc'))+max_mother_visit)

### 2. Criando Id sequencial do paciente - Mãe (PersonId)

In [11]:
window_person_id = Window.partitionBy().orderBy('id_cidacs_mae_sinasc')

df_person_id = (df
                .select("id_cidacs_mae_sinasc")
                .distinct()
                .withColumn('person_id', F.row_number().over(window_person_id))
               )

print(f'Quantidade de person_ids distintos: {df_person_id.count()}')
print(f'Colunas de/para person_id: {df_person_id.columns}')

Quantidade de person_ids distintos: 12671004
Colunas de/para person_id: ['id_cidacs_mae_sinasc', 'person_id']


In [12]:
# Adicionando o person_id com left join à base completa
df = df.join(df_person_id, how='left', on='id_cidacs_mae_sinasc')

### 2.1 Criando Id sequencial infant (PersonID do filho)

- Para criar o person_id do infant, vamos pegar o maior person_id da mãe e somar com a sequencia criada para o filho 

In [13]:
# Encontrando o maior person_id da mãe
max_mother_person_id = df.agg(F.max(F.col('person_id')).alias('max_value')).collect()[0]['max_value']
max_mother_person_id

25/05/06 13:57:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:57:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:57:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:57:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:57:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:57:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 1

12671004

In [14]:
window_person_id_infant = Window.partitionBy().orderBy('id_cidacs_sinasc_v5')

df_person_id_infant = (df
                .select("id_cidacs_sinasc_v5")
                .distinct()
                .withColumn('person_id_infant', F.row_number().over(window_person_id_infant) + max_mother_person_id)
               )

print(f'Quantidade de person_ids distintos da criança: {df_person_id_infant.count()}')

Quantidade de person_ids distintos da criança: 16588298


### Observação sobre person_id e person_id_infant
- A quantidade de infant deveria ser igual ou muito próxima da quantidade de registros da base completa, pois em teoria cada linha representa um nascimento.
- As causas dessas divergências podem ser duplicidade de registros ou gêmeos
- 24707761 linhas na basa
- 24695595 de infants distintos na base

In [15]:
# coletando o menor person_id_infant para validar se corresponde ao valor do maior person_id da mãe +1
print(f"Menor person_id_infant: {df_person_id_infant.agg(F.min(F.col('person_id_infant')).alias('min_value')).collect()[0]['min_value']}")

25/05/06 13:58:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Menor person_id_infant: 12671005


In [16]:
# Adicionando o person_id_infant a base completa através do left join pelo id_cidacs_sinasc_v4
df = df.join(df_person_id_infant, how='left', on='id_cidacs_sinasc_v5')
print(f'Quantidade de registros após à adição do person_id_infant: {df.count()}')

Quantidade de registros após à adição do person_id_infant: 16609493


In [17]:
# verificando se existe algum person_id_infant nulo no novo dataframe
print(f"Quantidade de person_id_infant nulos: {df.filter(F.col('person_id_infant').isNull()).count()}")

25/05/06 13:58:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 13:58:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 1

Quantidade de person_id_infant nulos: 0


### 3. Criando LocationID

Observações:
1. Não tem codmunres_sinasc null
2. dtnasc_sinasc não tem valores nulos

Como existem pessoas (person_id) que tem mais de um 'codmunres_sinasc' (CEP), vamos pegar o último 'codmunres_sinasc' não nulo ordenando pela data de nascimento do filho (dtnasc_sinasc) em ordem crescente

In [18]:
window_location_id = Window.partitionBy('person_id').orderBy('dtnasc_sinasc')

df = df.withColumn('location_id', F.last('codmunres_sinasc', ignorenulls=True).over(window_location_id))

### 4. Calculando Data de nascimento da mãe  

- Quando a data de nascimento da mãe (dtnascmae_sinasc) for nulo, fazer o cálculo: dtnasc_sinasc - idademae_sinasc.

In [19]:
window_dt_mae = Window.partitionBy('id_cidacs_mae_sinasc').orderBy('dtnasc_sinasc').rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

In [20]:
df = (df.withColumn('dt_nascimento_calc_mae', F.when(F.col('dtnascmae_sinasc').isNotNull(), 
                                                 F.to_date('dtnascmae_sinasc', 'yyyy-MM-dd'))
                                           .when(F.col('dtnascmae_sinasc').isNull(), 
                                                F.add_months(F.to_date('dtnasc_sinasc', 'yyyy-MM-dd'), -12 * F.col('idademae_sinasc').cast('integer')))
                                           .otherwise(None).cast('date')
                                          )
      .withColumn('dt_nascimento_calc_mae', F.max('dt_nascimento_calc_mae').over(window_dt_mae))
     )

### 4.1 Removendo casos em que o dtnascmae_sinasc e idademae_sinasc são nulos

Onde o dtnascmae_sinasc é nulo, não é possível calcular ou extrair o ano de nascimento da mãe, e este é um campo obrigatório para o OMOP.

O que fazer?
R: Como existem poucos registros onde a data de nascimento da mãe é nulo depois de calcular o item 4, vamos manter apenas os not nulls.

In [21]:
df = df.filter(F.col('dt_nascimento_calc_mae').isNotNull())
print(f'Quantidade de registros onde é possível extrair o ano de nascimento da mãe: {df.count()}')

Quantidade de registros onde é possível extrair o ano de nascimento da mãe: 16609485


### Salvando Base completa enriquecida no formato Parquet

In [22]:
(df
 .repartition(1)
 .write
 .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/basefinal_gest_limp_enriched', mode='overwrite')
)

25/05/06 14:00:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 14:00:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 14:00:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 14:00:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 14:00:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 14:00:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/06 1